In [3]:
#!pip install -U llamaapi

In [4]:
# Import necessary librearies


import inspect
import re
import os
from dotenv import load_dotenv
load_dotenv()

from getpass import getpass
from langchain import OpenAI, PromptTemplate
from langchain .chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.callbacks import get_openai_callback

In [5]:
llm = OpenAI(
    temperature=0,
    openai_api_base=os.getenv('OPENAI_REVERSE_PROXY'),
    openai_api_key=os.getenv('OPENAI_API_KEY')
)

In [6]:
# As a responsible gesture we will count the number of tokens everytime we call 

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result  = chain.run(query)
        print(f"Total number of tokens is {cb.total_tokens} tokens")

In [7]:
#Mainly two types of chain are there
#Generic chains, Utility chains, and User defined chains

#Generic chains
#Utility chains
#User defined chains

#Generic chains are the ones which are already defined in the framework
#Utility chains are the ones which serve a special purpose eg mathchains
#User defined chains are the ones which are defined by the user 


math_chain = LLMMathChain(llm=llm,verbose=True)

count_tokens(math_chain,"What is cubed value of the value of pi")

/home/susearc/gamedisk/stable-torch/lib/python3.9/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new  chain...
What is cubed value of the value of pi```text
3.141592653589793**3
```
...numexpr.evaluate("3.141592653589793**3")...

Answer: 31.006276680299816
> Finished chain.
Total number of tokens is 272 tokens


In [8]:
print(math_chain.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [10]:
prompt = PromptTemplate(input_variables=['question'], template='{question}')

llm_chain = LLMChain(prompt = prompt, llm=llm)


In [12]:
count_tokens(llm_chain, "what is 12 to the power 0.4453?")

Total number of tokens is 17 tokens


In [14]:
print(inspect.getsource(math_chain._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



In [15]:
# Transform text in order to get cleaned text as input in order to avoid unnecessary token usage

def transform_func(inputs: dict) -> dict:
    text = inputs['text']

    # replace multiple lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [16]:
clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)

In [18]:

#example of cleaned up text

clean_extra_spaces_chain.run('A random text  with   some irregular spacing.\n\n\n     Another one   here as well.')

'A random text with some irregular spacing.\n Another one here as well.'

In [19]:
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """
prompt = PromptTemplate(input_variables=["style", "output_text"], template=template)

In [21]:
style_chain = LLMChain(llm=llm, prompt=prompt, output_key='final_output')


In [25]:


seq_chain = SequentialChain(chains=[clean_extra_spaces_chain, style_chain], input_variables=['text', 'style'], output_variables=['final_output'])

In [26]:
input_text = """
Chains allow us to combine multiple 


components together to create a single, coherent application. 

For example, we can create a chain that takes user input,       format it with a PromptTemplate, 

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by 


combining chains with other components.
"""

In [29]:
count_tokens(seq_chain, {"text":input_text, "style":'a 90s rapper'})

Total number of tokens is 163 tokens
